# Kryptologie LAB - CBC

In [114]:
import numpy as np
import random

## AES

In [115]:
# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [116]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox")
    print(sbox)

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox Inverse")
    print(sbox_inverse)

def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")

def blockToText(textblock):
    text = textblock.transpose().flatten()
    toChar = lambda c: chr(c)
    return "".join(np.array([toChar(c) for c in text]))

def addRoundKey(textblock, key):
    return np.bitwise_xor(textblock, key)

def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            row = (textblock[y,x] >> 4) & 0xF
            col = textblock[y,x] & 0xF
            textblock[y,x] = box[row, col]
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

def encryptBlock(textblock, keys, inputAsMatrix=False, returnAsBlock=False):
    if not inputAsMatrix:
        textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    if returnAsBlock:
        return textblock
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

SBox
[[ 99 124 119 123 242 107 111 197  48   1 103  43 254 215 171 118]
 [202 130 201 125 250  89  71 240 173 212 162 175 156 164 114 192]
 [183 253 147  38  54  63 247 204  52 165 229 241 113 216  49  21]
 [  4 199  35 195  24 150   5 154   7  18 128 226 235  39 178 117]
 [  9 131  44  26  27 110  90 160  82  59 214 179  41 227  47 132]
 [ 83 209   0 237  32 252 177  91 106 203 190  57  74  76  88 207]
 [208 239 170 251  67  77  51 133  69 249   2 127  80  60 159 168]
 [ 81 163  64 143 146 157  56 245 188 182 218  33  16 255 243 210]
 [205  12  19 236  95 151  68  23 196 167 126  61 100  93  25 115]
 [ 96 129  79 220  34  42 144 136  70 238 184  20 222  94  11 219]
 [224  50  58  10  73   6  36  92 194 211 172  98 145 149 228 121]
 [231 200  55 109 141 213  78 169 108  86 244 234 101 122 174   8]
 [186 120  37  46  28 166 180 198 232 221 116  31  75 189 139 138]
 [112  62 181 102  72   3 246  14  97  53  87 185 134 193  29 158]
 [225 248 152  17 105 217 142 148 155  30 135 233 206  85

In [117]:
def subByte(byte, box):
    row = (byte >> 4) & 0xF
    col = byte & 0xF
    return box[row, col]

def rotWord(word):
    return np.roll(word, shift=-1)

def subWord(word):
    return [subByte(byte, sbox) for byte in word]

def rcon(i):
    array = [0x1, 0x2, 0x4, 0x8, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]
    return [array[i], 0,0,0]

# input 16 character string
def generateKey(key):
    block = initBlock(key)
    words = []
    words.append(block[:,0])
    words.append(block[:,1])
    words.append(block[:,2])
    words.append(block[:,3])
    for i in range(4, 44):
        if i % 4 == 0:
            val = gfAddition(words[i-4], rcon(i // 4 - 1))
            val = gfAddition(subWord(rotWord(words[i-1])), val)
            words.append(val)
        else:
            words.append(gfAddition(words[i-4], words[i-1]))
    keys = np.empty((11, 4,4), dtype=np.dtype('B'))
    for i in range(0, 44):
        keys[i // 4, :, i % 4] = words[i]
    return keys

## CBC

In [118]:
blockLength = 16
IV = np.zeros((4,4), dtype=np.dtype('B'))
keys = generateKey("EINSCHLUESSEL123")

In [119]:
def getChunks(text, blockLength=16):
    return [text[i:i+blockLength] for i in range(0, len(text), blockLength)]

def CBCMAC(k, chunks):
    xorVec = IV
    output = ""
    for chunk in chunks:
        chunkAsBytes = initBlock(chunk)
        inputBytes = np.bitwise_xor(chunkAsBytes, xorVec)
        xorVec = encryptBlock(inputBytes, k, inputAsMatrix=True, returnAsBlock=True)
        output = xorVec
    return blockToText(output)

In [120]:
text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
chunks = getChunks(text, blockLength)
print("Chunks:\n")
print(chunks)
CBCMAC(keys, chunks)

Chunks:

['Lorem ipsum dolo', 'r sit amet, cons', 'etetur sadipscin', 'g elitr, sed dia', 'm nonumy eirmod ', 'tempor invidunt ', 'ut labore et dol', 'ore magna aliquy', 'am erat, sed dia', 'm voluptua. At v', 'ero eos et accus', 'am et justo duo ', 'dolores et ea re', 'bum. Stet clita ', 'kasd gubergren, ', 'no sea takimata ', 'sanctus est Lore', 'm ipsum dolor si', 't amet. Lorem ip', 'sum dolor sit am', 'et, consetetur s', 'adipscing elitr,', ' sed diam nonumy', ' eirmod tempor i', 'nvidunt ut labor', 'e et dolore magn', 'a aliquyam erat,', ' sed diam volupt', 'ua. At vero eos ', 'et accusam et ju', 'sto duo dolores ', 'et ea rebum. Ste', 't clita kasd gub', 'ergren, no sea t', 'akimata sanctus ', 'est Lorem ipsum ', 'dolor sit amet.']


'Ø¾¼É¤É±§Bºe4ÊìõU'

## CCM-Modus

In [130]:
def initBlockFromInt(number: int):
    bytes = []
    for byte in range(15,-1,-1):
        bytes.append((number >> (8 * byte)) & 0xFF)
    return np.array(bytes, dtype=np.uint8).reshape((4,4)).transpose()

def counter(k, nonce, inputChunks):
    output = ""
    init_ctr = nonce << 64
    # Counter encryption
    for i in range(len(inputChunks)):
        chunkAsBytes = initBlock(inputChunks[i])
        T_i = (init_ctr + i + 1) % (2 ** 128)
        T_i_matrix = initBlockFromInt(T_i)
        encryptedT = encryptBlock(T_i_matrix, k, inputAsMatrix=True, returnAsBlock=True)
        output += blockToText(np.bitwise_xor(chunkAsBytes, encryptedT))
    return output

def encryptAndSignCCM(k, nonce, inputChunks):
    output = counter(k, nonce, inputChunks)
    # CBCMac
    hash = CBCMAC(k, chunks)
    output += hash
    return output

def decryptAndVerify(k, nonce, inputChunks):
    message = counter(k, nonce, inputChunks[:-1])
    print("Message:\n", message)
    # CBCMac
    #hash = CBCMAC(k, getChunks(message))
    #expectedHash =  chunks[-1]
    #print("exctracted: ",expectedHash)
    #assert hash == expectedHash
    return message

In [123]:
nonce = random.getrandbits(64)
print(nonce)

17051384817711973308


In [133]:
# Test
chunks = getChunks(text, blockLength)
output = counter(keys, nonce, chunks)
print(output)
print(counter(keys, nonce, getChunks(output, blockLength)))

ÆÊÛÉK¬V«©Ö3¨ýbÏ&CxµÚß6<ï\!é]'ìä?²pñÁ©6Ê¶B¬o5/Õ«Æ^UY3þ7éèE$lï×.(³¬üøtuñR¶fßlKàBÌRrÉ
JçÕÃñÎh}½?nXÁà;Òié[qRêÓ?Açã^¤Úi°%{¡Áb¹ü`kR6ª?§ ËaàÉÞm«&èáu;Ü,±²Á9U¨¦TRB¸WQÕ%ÂwÏæñY=z}öéEupÕ]p@Bku	ZE]¤á¦íØgRBÝôF-Sdï¡k+á&#F-¬ß¦¶ëÛ@Ç4
½0
ýl/nÊ½2õþ~v1«»Ã÷B¹s5òb?-Fax3ì¦.¬õÞDDaÊoWF_è ºòÓn]X"ãþØq/Ó,¤äD±áþÿâÍJy
$Ì<¡ôÙPªÕªTÚZeo½O®hºåB©8Øv¡?¿õj¼wïç¥vªK-ª©²ùÆÄÁ6ÿ§«£¤>ZïÖ3çñ°Ó(=_Ó>"éb¡ÐáÌÕª¹4½ÕÎé1ß­v=Ý1¸#A+Ê[¹#¸eh¥()O«ïÔä;ÓÎP¶°6ÄLIÞ_7ê-ý|cõw«E'FéWýzù+&FYô&Ç|¹è<Î§ôAm6Ï#ÃÇü·ä:¯ÃÁ§§Ájæ­¶=;5ÄDuCÿãEèð
Ý¥¾y0


In [131]:
text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
chunks = getChunks(text, blockLength)

y = encryptAndSignCCM(keys, nonce, chunks)
y

"ÆÊÛÉK¬V«©Ö3¨\x87ýbÏ&CxµÚß6<ï\\!é]'ìä?\x14²pñÁ©\x9e\x03\x03\x876Ê¶B\x14¬o5/Õ\x7f«Æ^UY\x953\x97þ\x967é\x0b\x19è\x19\x81E$lï×.(³¬üøt\x0bu\x98ñR\x8d¶fßlKà\x01B\x7fÌ\x87RrÉ\n\x024G\x04q¸å|Õ¾¾Æ\x1b?8 $þãì\xa0\x82ß\x17°ò÷þK\x97ý>Iøÿ\x1bËËV°:dÿD£àaJ:º]´~__¶1\x9dÊK*É÷\x07TvÓ\x19\x84Oì\rJç\x91ÕÃñÎh}½\x8b?\x88nXÁà;Òié[qRêÓ?Aç\x9b\x07\x9fã^\x91\x03¤Úi°\x86%{\x90\x9a¡Á\x86\x88b¹ü`\x0e\x92kR6ª?§ Ëaà\x80ÉÞm«&èá\x8eu\x8f;\x95Ü\x7f\x12,\x89±\x83²Á9\x1fU¨\x82¦TRB¸WQÕ%ÂwÏ\x15æ\x18ñY=\x8cz}öéE\x1aupÕ]p@Bku\tZ\x92\x8fE]¤á¦\x95\x89íØgR\x9fBÝô\x85F-\x81S\x0bdï¡k+á&#F\x8f-¬ß¦¶ëÛ@Ç4\n©d\x02\x18\x88e8\x92²¢\x01\x99\tÖÚö\x1aëå( \x0fàEU\\\x90C[\x90D\x97\x88ÙPû&\x83äèýÁ±w\x1dw®fÐ\x08È¢[ý8\x88vÅD\x03\x0e\x08qt\x8a\x06\x8c2v\x9fD\x1c\x11ÄÙïv\x9còL=\x98\x06\x97\x9aû\x1b\x9cÃÞh)cÈ íhEH¶\x89\x1d%|\x92¸WÕ\x02+Âõ\x11Ë\x8c\x08E\x07¸JgOËó]\x9bÈ.LÓ\x07s\x16èÃ¼¦\x1d¶\x03R\x96QWó=rÁ\x98ùd`µI_ÛY3mÂ\x93MäÛ\x84\x98ð&aGÔ\x93\x9f8\x80±V\x08\x08©ÊQI\x9döËCzä\x1d\x8b5Ob6æ%ý\x1eærF\\ÕÂàÎ\x018\x89Y\x1bò9ì\x9c&Ë98!\rýþoC¥\x1aOþZ\x89\x

In [125]:
message = decryptAndVerify(keys, nonce, getChunks(y))

[[138  38 218 227]
 [165 140 163 146]
 [169  63  94  14]
 [172 219 136 160]]
[[ 84 174 138  62]
 [ 99 255  40  72]
 [ 11  87  13 130]
 [220  81 201 151]]
[[ 90 132 255  69]
 [ 96 179 103 169]
 [215  32 106 223]
 [  4 218 247  44]]
[[115  70 230 181]
 [140 161  45  87]
 [ 10  13  48 254]
 [ 89 135  61 159]]
[[251 119 101 186]
 [ 23 157  65  65]
 [135 116   5  76]
 [100 248 157 147]]
[[216 100  60 170]
 [153   7 251   2]
 [149 184 223  63]
 [  4 152   2 192]]
[[116 230 111  36]
 [ 54  48  34  21]
 [ 95  29  81 215]
 [160 187  51 137]]
[[ 19 171  65 133]
 [167 122   4 209]
 [219  88 159 247]
 [158  86 143 166]]
[[118 140  30  59]
 [221  42  58 175]
 [210 227 157 162]
 [146 209 155  55]]
[[221  40  63 245]
 [ 26 214  91  10]
 [ 18 144 148 127]
 [144  21 125  41]]
[[211  46  98 122]
 [ 67  69  32 231]
 [242 186  86  58]
 [234 215 178 159]]
[[108 161  27  91]
 [ 39 227   9 253]
 [199 155 210   1]
 [244 187 171 120]]
[[165  27  55  32]
 [143 140 158 199]
 [ 87  40 243 233]
 [189  81  90  98]]